In [1]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import time

In [2]:
def tong_list(number):
    url = 'https://tw.stock.yahoo.com/quote/'
    web = requests.get(url+ number)
    #time.sleep(1)
    #print(web.text)
    #print(web.content)

    soup = BeautifulSoup(web.content, 'html.parser')# 解析網頁內容
    my_list = []# 打印結果


    ''' 第一段開始 '''
    my_list.append(number)
    elements_1 = soup.find(class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px)")
    #print(elements_2)
    #numberstr = str(elements_2.text).replace('收盤 | ','')
    #print(elements_1.text)
    my_list.append(str(elements_1.text))
    ''' 第一段結束 '''


    ''' 第二段開始 '''
    elements_2 = soup.find(class_="C(#6e7780) Fz(12px) Fw(b)")
    #print(elements_2)
    numberstr = str(elements_2.text).replace('收盤 | ','')
    numberstr = numberstr.replace(' 更新', '')
    #print(numberstr)
    my_list.append(numberstr)
    ''' 第二段結束 '''

    ''' 第三段開始 '''

    target_class_string = "Fw(600) Fz(16px)--mobile Fz(14px)"  # 定義目標的 class 字串

    # 自定義篩選函數，檢查元素的 class 屬性是否包含這整個字串
    def has_target_class_string(tag):
        class_attr = " ".join(tag.get('class', []))  # 將 class 列表轉換為字串
        #print(class_attr) ## 這把所有的class 找出來耶。
        return target_class_string in class_attr  ## 多思，他這裡是還有去做條件， 去把符合的class給return

    # 使用自定義篩選函數查找包含目標 class 字串的元素
    elements_3 = soup.find_all(has_target_class_string)
    #print(elements_1.text)
    for element in elements_3:
        #sprint(element.text)
        try:
            number = float(element.text)
        except:
            if ',' in str(element.text):
                numberstr = str(element.text).replace(',','')
            if '%' in str(element.text):
                numberstr = str(element.text).replace('%','')
            number = float(numberstr)
        my_list.append(number)

    ''' 第三段結束 '''




    return my_list

In [3]:
# 建立要抓取的股票網址清單
stock_urls = ['2330','0050','2317','6547']
#stock_urls = ['2330']
my_list_list = []
for i in stock_urls:
    #print(i)
    mylist = tong_list(i)
    my_list_list.append(mylist)

In [4]:
columns = ['股票代號','公司簡稱','資料更新時間','成交','開盤','最高','最低','均價','成交金額(億)','昨收','漲跌幅%','漲跌','總量','昨量','振幅%']
my_df = pd.DataFrame(my_list_list, columns=columns)
#print(my_df)
# 顯示股票資訊
display(my_df)

,股票代號,公司簡稱,資料更新時間,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅%,漲跌,總量,昨量,振幅%
0,2330,台積電,2024/09/20 14:30,973.00,981.0,982.0,971.00,975.00,585.830,960.00,1.35,13.0,60066.0,32674.0,1.15
1,0050,元大台灣50,2024/09/20 14:30,182.45,183.1,184.1,182.45,183.09,24.930,180.15,1.28,2.3,13617.0,8081.0,0.92
2,2317,鴻海,2024/09/20 14:30,177.00,180.0,182.0,177.00,179.20,146.010,176.50,0.28,0.5,81476.0,41427.0,2.83
3,6547,高端疫苗,2024/09/20 13:30,46.35,47.5,47.5,46.30,46.60,0.302,47.05,1.49,0.7,648.0,341.0,2.55
